# Imports

In [1]:
import numpy as np
import pandas as pd
import os

from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo = False)

In [2]:
dfcustomers = pd.read_csv('input/olist_customers_dataset.csv')
dfgeolocation = pd.read_csv('input/olist_geolocation_dataset.csv')
dforder_items = pd.read_csv('input/olist_order_items_dataset.csv')
dforder_payments = pd.read_csv('input/olist_order_payments_dataset.csv')
dforder_reviews = pd.read_csv('input/olist_order_reviews_dataset.csv')
dforders = pd.read_csv('input/olist_orders_dataset.csv')
dfproducts = pd.read_csv('input/olist_products_dataset.csv')
dfsellers = pd.read_csv('input/olist_sellers_dataset.csv')
dfcategory_name = pd.read_csv('input/product_category_name_translation.csv')

dfcustomers.to_sql('customers', con=engine)
dfgeolocation.to_sql('geolocation', con=engine)
dforder_items.to_sql('order_items', con=engine)
dforder_payments.to_sql('order_payments', con=engine)
dforder_reviews.to_sql('order_reviews', con=engine)
dforders.to_sql('orders', con=engine)
dfproducts.to_sql('products', con=engine)
dfsellers.to_sql('sellers', con=engine)
dfcategory_name.to_sql('category_name', con=engine)

71

# Scripts

### En excluant les commandes annulées, quelles sont les commandes récentes de moins de 3 mois que les clients ont reçues avec au moins 3 jours de retard ?

In [3]:
query = '''
WITH max_date AS (
    SELECT MAX(order_purchase_timestamp) AS last_order_date
    FROM orders
)
SELECT 
    o.order_id,
    o.customer_id,
    o.order_purchase_timestamp,
    o.order_delivered_customer_date,
    JULIANDAY(o.order_delivered_customer_date) - JULIANDAY(o.order_estimated_delivery_date) AS days_late
FROM 
    orders o, max_date m
WHERE 
    (o.order_status = 'delivered' OR o.order_status = 'shipped') 
    AND o.order_delivered_customer_date IS NOT NULL
    AND JULIANDAY(m.last_order_date) - JULIANDAY(o.order_purchase_timestamp) <= 90  
    AND JULIANDAY(o.order_delivered_customer_date) - JULIANDAY(o.order_estimated_delivery_date) >= 3 
'''
pd.read_sql_query(query, con=engine)

,order_id,customer_id,order_purchase_timestamp,order_delivered_customer_date,days_late
0,cfa4fa27b417971e86d8127cb688712f,7093250e1741ebbed41f0cc552025fd6,2018-08-16 09:44:23,2018-08-29 01:41:41,7.070613
1,234c056c50619f48da64f731c48242b4,44e460a655f7154ccd9faa4dbbbaf68a,2018-08-14 14:49:15,2018-09-01 18:14:42,9.760208
2,7f579e203c931f3e8410103359c6d523,d665be250d1c687c58fdea61a9b55a58,2018-08-02 18:09:27,2018-08-13 20:11:47,4.841516
3,cb6e441ff2ef574ce08d3709426f88ec,4fb843d304c57182d4aa27bb39ca592b,2018-08-08 19:27:03,2018-08-18 01:11:58,3.049977
4,03720fdc92032ee4abd471d172006ab0,116458665bac0ff47d5e87f65e8ec681,2018-08-05 21:34:54,2018-08-21 00:11:52,4.008241
...,...,...,...,...,...
302,f6ac44bbece0129b596dfc21aede3c20,c7616d00b4400791c265c3b952dee75e,2018-08-07 14:07:40,2018-08-21 18:07:43,7.755359
303,874a7690bc049bd4ce210d195bdfff7b,f6b2c83ea477af36f44a1e2df7320036,2018-08-02 12:06:47,2018-08-13 15:50:48,5.660278
304,587e32dd528769d669d539531d32aeb5,0af8a25fb0b6f833724bb5818275807b,2018-08-10 11:46:09,2018-09-03 09:32:31,6.397581
305,3adb141ba4bd69dd7fe8d3fb733c6b74,c0539d5c87fc7c97a8418adffe4b45f0,2018-08-14 23:29:21,2018-08-28 18:02:52,4.751991


### Qui sont les vendeurs ayant généré un chiffre d'affaires de plus de 100 000 Real sur des commandes livrées via Olist ?

In [4]:
query = '''
SELECT
  seller_id,
  SUM(oi.price) AS total_revenue
FROM
  order_items oi
JOIN
  orders o ON oi.order_id = o.order_id
WHERE
  o.order_status = 'delivered'
GROUP BY
  seller_id
HAVING
  SUM(oi.price) > 100000
ORDER BY
  total_revenue DESC;
'''
pd.read_sql_query(query, con=engine)

,seller_id,total_revenue
0,4869f7a5dfa277a7dca6462dcf3b52b2,226987.93
1,53243585a1d6dc2643021fd1853d8905,217940.44
2,4a3ca9315b744ce9f8e9374361493884,196882.12
3,fa1c13f2614d7b5c4749cbc52fecda94,190917.14
4,7c67e1448b00f6e969d365cea6b010ab,186570.05
5,7e93a43ef30c4f03f38b393420bc753a,165981.49
6,da8622b14eb17ae2831f4ac5b9dab84a,159816.87
7,7a67c85e85bb2ce8582c35f2203ad736,139658.69
8,1025f0e2d44d7041d6cf58b6550e0bfa,138208.56
9,955fee9216a65b617aa5c0531780ce60,131836.71


### Qui sont les nouveaux vendeurs (moins de 3 mois d'ancienneté) qui sont déjà très engagés avec la plateforme (ayant déjà vendu plus de 30 produits) ?

In [7]:
query = '''
WITH max_date AS (
    SELECT MAX(order_purchase_timestamp) AS last_order_date
    FROM orders
)
SELECT 
    seller_id,
    MIN(o.order_purchase_timestamp) AS first_sale_date,
    COUNT(oi.product_id) AS total_products_sold
FROM 
    order_items AS oi
JOIN 
    orders AS o ON oi.order_id = o.order_id
JOIN
    max_date AS m
WHERE 
    o.order_status = 'delivered'
GROUP BY 
    seller_id
HAVING 
    JULIANDAY(m.last_order_date) - JULIANDAY(MIN(o.order_purchase_timestamp)) <= 90 
    AND COUNT(oi.product_id) > 30
ORDER BY 
    total_products_sold DESC

'''
pd.read_sql_query(query, con=engine)

,seller_id,first_sale_date,total_products_sold
0,d13e50eaa47b4cbe9eb81465865d8cfc,2018-08-04 09:09:37,68
1,81f89e42267213cb94da7ddc301651da,2018-08-08 12:45:12,52


### Quels sont les 5 codes postaux, enregistrant plus de 30 reviews, avec le pire review score moyen sur les 12 derniers mois ?

In [6]:
query = '''
WITH max_order_date AS (
    SELECT MAX(o.order_purchase_timestamp) AS last_order_date
    FROM orders o
)
SELECT 
    c.customer_zip_code_prefix AS postal_code,
    COUNT(r.review_id) AS total_reviews,
    AVG(r.review_score) AS avg_review_score
FROM 
    order_reviews AS r
JOIN 
    orders AS o ON r.order_id = o.order_id
JOIN 
    customers AS c ON o.customer_id = c.customer_id
JOIN
    max_order_date AS m
WHERE 
    JULIANDAY(m.last_order_date) - JULIANDAY(r.review_creation_date) <= 365
GROUP BY 
    c.customer_zip_code_prefix
HAVING 
    COUNT(r.review_id) > 30
ORDER BY 
    avg_review_score ASC
LIMIT 5;

'''
pd.read_sql_query(query, con=engine)

,postal_code,total_reviews,avg_review_score
0,22753,47,2.808511
1,22770,37,3.135135
2,22793,93,3.247312
3,13056,31,3.290323
4,13295,32,3.312500
